In [27]:
import numpy as np
import matplotlib.pyplot as plt

def calculate_qxy(x, lambda_, pixel_size, pos_direct_x, D, PONI_x, delta):
    # Calculate the position of the direct beam on the rotated detector
    pos_direct_x = PONI_x - (D / pixel_size) * np.tan(np.radians(delta))
    
    # Calculate the angle theta for each pixel with horizontal coordinate x
    theta = np.arctan(pixel_size * (x - pos_direct_x) / D)/2.
    
    # Calculate q_xy
    qxy = (4 * np.pi / lambda_) * np.sin(theta)
    
    return qxy

# Function to load the data from a CSV file (one-line header, space-separated)
def load_data(file_path):
    # Load the file with numpy, assuming it's space-separated and has a single-line header
    data = np.loadtxt(file_path, delimiter=None, skiprows=1)
    
    # The first column represents pixel values and the second column represents intensity values
    x_values = data[:, 0]  # Column 0 as pixel values
    intensity_values = data[:, 1]  # Column 1 as intensity values
    
    return x_values, intensity_values

# Main function to process the data and convert pixel values to q_xy
def process_file(file_path, lambda_, pixel_size, D, PONI_x, delta):
    # Load the data from the CSV file
    x_values, intensity_values = load_data(file_path)
    
    # Calculate q_xy for each pixel value
    qxy_values = np.array([calculate_qxy(x, lambda_, pixel_size, 0, D, PONI_x, delta) for x in x_values])

    # Combine the results into a structured array or simply return the qxy_values alongside the intensity
    processed_data = np.column_stack((x_values, qxy_values, intensity_values))
    
    return processed_data

# Example values (replace with your actual values)
lambda_ = 1.24e-10  # Wavelength in meters
pixel_size = 172e-6  # Pixel size in meters
D = 0.35  # Distance between detector and sample in meters
PONI_x = 563.5  # Example value for PONI_x
delta = -8.7  # In-plane angle in degrees

# Path to the CSV file (replace with your actual file path)
file_path = '5151-pixel-integration-horizontal.txt'

# Process the file and get the data with q_xy values
processed_data = process_file(file_path, lambda_, pixel_size, D, PONI_x, delta)

# Convert to A^-1
processed_data[:,1]=processed_data[:,1]*1e-10

# Print or save the processed data
print(processed_data)

# Optionally save the results to a new file
np.savetxt('processed_data.csv', processed_data, delimiter=';', header="Pixel;q_xy(A^-1);Intensity", comments='', fmt='%f')


[[ 1.00000000e+00 -2.04131889e+00  1.26352400e+03]
 [ 2.00000000e+00 -2.03925920e+00  1.34314300e+03]
 [ 3.00000000e+00 -2.03719868e+00  1.37595200e+03]
 ...
 [ 8.98000000e+02  5.75661600e-02  9.74104800e+03]
 [ 8.99000000e+02  6.00558929e-02  9.86314300e+03]
 [ 9.00000000e+02  6.25455932e-02  1.09177100e+04]]
